In [1]:
# USER-SPECIFIC: The folder paths holding simulation outputs and observations

if not 'Simulation_output_folders' in locals():  
    Simulation_output_folders = [
        r'/Users/admin/Desktop/OneDrive - IIASA/Papers/2024_Refining_humans_CWatM/Results/output_5min_Bhima_1979_2016',
        #r'/Users/admin/Desktop/OneDrive - IIASA/Papers/2024_Refining_humans_CWatM/Results/output_5min_Bhima_1979_2016',
        #r'/Users/admin/Desktop/OneDrive - IIASA/Papers/2024_Refining_humans_CWatM/Results/Bhima_01011979_31122016_StdReservoirs2',
        #r'/Users/admin/Desktop/OneDrive - IIASA/Papers/2024_Refining_humans_CWatM/Results/Bhima_01011979_31122016',    
    ]
if not 'titles' in locals():
    titles = ['observations']+['dataset '+str(num+1) for num in range(len(Simulation_output_folders))]
else: 
    titles = ['observations'] + titles
    
if not 'output_variable' in locals():
    output_netcdf = 'discharge_daily'
    output_variable = 'discharge'
    
# observations_folder holds one excel (observation_locations.xlsx) with names and locations, and
    # a folder (Observations) with an Excel file for each station. The Excel files are named after the station names.
    #
    # observations_locations.xlsx has three columns: Station, Latitude, Longitude
    #
    # observations_folder
    # ↵ observations_locations.xlsx
    # ↵ Observations
    #   ↵ StationName1.xlsx #first two rows not read
    #   ↵ StationName2.xlsx #first two rows not read
    #   ↵ ...
if not 'template' in locals():
     template = "plotly_dark"
if not 'observations_folder' in locals():      
    observations_folder = '/Users/admin/Desktop/OneDrive - IIASA/Papers/2024_Refining_humans_CWatM/Observations/River water level and discharge_Historical'

In [2]:
from netCDF4 import Dataset, num2date
import datetime
#from datetime import datetime
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [3]:
#observed_discharge_folder = observations_folder + '/Observations' #inside functions
Simulated_output_paths = [folder + '/' + output_netcdf + '.nc' for folder in Simulation_output_folders]

In [4]:
def geo_idx(dd, dd_array):
   """
     search for nearest decimal degree in an array of decimal degrees and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
    """
   geo_idx = (np.abs(dd_array - dd)).argmin()
   return geo_idx

### Observed discharge

In [5]:
from functions import read_observations_excel

Stations_namesLocations, DATES_observed, FLOWS_observed = read_observations_excel(observations_folder=observations_folder)

Loading observations
Loading observations


KeyError: 'Date'

### Simulated discharge

In [ ]:
NC_SIMULATED = []
LATS = []
LONS = []
DATES_SIMULATED = []
OUTPUTS_SIMULATED = []

print('Loading simulations')
for simulation in Simulated_output_paths:
    
    nc_simulated = Dataset(simulation, 'r')
    lats = nc_simulated.variables['lat'][:]
    lons = nc_simulated.variables['lon'][:]
    Dates_simulated = num2date(nc_simulated.variables['time'][:], units=nc_simulated.variables['time'].units, 
                               only_use_cftime_datetimes=False, only_use_python_datetimes=True)
    
    Dates_simulated = pd.to_datetime(Dates_simulated).date

    Outputs_simulated = []
    
    NC_SIMULATED.append(nc_simulated)
    LATS.append(lats)
    LONS.append(lons)
    DATES_SIMULATED.append(Dates_simulated)
    OUTPUTS_SIMULATED.append(Outputs_simulated)

for discharge_location in Stations_namesLocations:
    in_lat = discharge_location[1]
    in_lon = discharge_location[2]
    
    for sim in range(len(Simulated_output_paths)):
        lats = LATS[sim]
        lons = LONS[sim]
    
        lat_idx = geo_idx(in_lat, lats)
        lon_idx = geo_idx(in_lon, lons)

        OUTPUTS_SIMULATED[sim].append(NC_SIMULATED[sim].variables[output_variable][:, lat_idx, lon_idx])


## Visualisation 

In [ ]:
RMSD_allStn, NRMSD_allStn, KGE_allStn, bias_allStn, corr_allStn, NS_allStn = [],[],[],[],[],[]

In [ ]:
import hydroStats as hydroStats
import math
for i in range(len(Stations_namesLocations)):

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(y=FLOWS_observed[i],
                             x=DATES_observed[i],
                    mode='lines+markers',
                    name=titles[0],
                            opacity=1, 
                             line=dict(width=1), #color='#1f77b4',
                            marker = dict(size=2)))
    
    RMSD, NRMSD, KGE, bias, corr, NS = [],[],[],[],[],[]

    for sim in range(len(Simulated_output_paths)):

        fig.add_trace(go.Scatter(y=OUTPUTS_SIMULATED[sim][i],
                                 x=DATES_SIMULATED[sim],
                        mode='lines',
                        name=titles[sim+1],
                                line=dict(width=1))) #color='#ff7f0e', 
    #"""
        FLOWS_simulated = OUTPUTS_SIMULATED[sim] 
        Dates_simulated = DATES_SIMULATED[sim]

        max_simulated = max(FLOWS_simulated[i])

        start = 0
        for d in range(len(DATES_observed[i])):
            # Find the match of the first observation to the first simulated day
            #print(DATES_observed[i][d])
            #print(Dates_simulated[0])
            if DATES_observed[i][d] == Dates_simulated[0]:
                start = d
                break
        startSim = 0
        if start == 0:
            #Simulation begins before observations
            #Find first simulation date
            for d in range(len(Dates_simulated)):
                if Dates_simulated[d] == DATES_observed[i][0]:
                    startSim = d
                    break

        if startSim == 0 and start ==0 and Dates_simulated[0] != DATES_observed[i][-1]:
            #print('There is no overlap of simulation and observation.')
            KGE.append('-')
            bias.append('-')
            corr.append('-')
            NS.append('-')
            RMSD.append('-')
            NRMSD.append('-')
            
        else:
            #print('d', d)

            oval=[]
            o = FLOWS_observed[i][start:]
            s = []

            end = 0
            for d in range(len(o)):
                if not o[d]=='' and not math.isnan(o[d]) : #or 0?
                    try:
                        if FLOWS_simulated[i][d+startSim] > 0:
                            s.append(FLOWS_simulated[i][d+startSim])
                            oval.append(o[d])
                    except: 
                        end = d
                        break  

            sval = np.asarray(s)
            MSE = 0
            for d in range(len(oval)):
                MSE += (oval[d]-sval[d])**2
            if len(oval) > 0:
                rmsd = (MSE/len(oval)) ** .5
            
                RMSD.append("{:.2f}".format(rmsd))
                NRMSD.append("{:.2f}".format(rmsd/np.mean(oval)))
                KGE.append("{0:.2f}".format(hydroStats.KGE(s=sval, o=oval, warmup=0)))
                bias.append("{0:.2f}".format(hydroStats.pc_bias2(s=sval, o=oval, warmup=0)))
                corr.append("{0:.2f}".format(hydroStats.correlation(s=sval, o=oval, warmup=0)))
                NS.append("{0:.2f}".format(hydroStats.NS(s=sval, o=oval, warmup=0)))
            else:
                RMSD.append('-')
                NRMSD.append('-')
                KGE.append('-')
                bias.append('-')
                corr.append('-')
                NS.append('-')

    data_WB = {'KGE': KGE, 'NS':NS, 'Bias':bias, 'Corr':corr, 'RMSD':RMSD, 'NRMSD':NRMSD}
    df = pd.DataFrame(data_WB)
    df=df.style.set_table_styles([{'selector': 'th', 'props': [('font-size', '12pt')]}]).set_properties(**{"font-size": "12pt"}).hide(axis='index') 
    
    #"""
    fig.update_layout(title=output_variable +': '+ Stations_namesLocations[i][0],
                   yaxis_title=output_variable, template=template)

    fig.show()
    
    print(Stations_namesLocations[i][0])
    display(df)
    
    RMSD_allStn.append(RMSD) 
    NRMSD_allStn.append(NRMSD)
    KGE_allStn.append(KGE)
    bias_allStn.append(bias)
    corr_allStn.append(corr)
    NS_allStn.append(NS)

In [ ]:
import plotly.express as px

z= [stn for stn in KGE_allStn]
titles = titles[1:]
fig = px.imshow(z, template=template, text_auto=True, zmin = -2, zmax = 1.0, title='Kling-Gupta efficiency',
                labels=dict(y="Station", x='Run', color="Rating"),
                x=titles,
                y=[i[0] for i in Stations_namesLocations])
#fig.show()

z= [stn for stn in KGE_allStn]
fig = px.imshow(z, template=template, text_auto=True, zmin = -2, zmax = 1.0, title='Kling-Gupta efficiency',
                labels=dict(y="Station", x='Run', color="Rating"),
                x=[i for i in range(len(Simulation_output_folders))],
                y=[i[0] for i in Stations_namesLocations])
fig.show()


z= [stn for stn in NS_allStn]
fig = px.imshow(z, template=template, text_auto=True, zmin = -2, zmax = 1.0, title='Nash-Sutcliffe efficiency',
                labels=dict(y="Station", x='Run', color="Rating"),
                x=[i for i in range(len(Simulation_output_folders))],
                y=[i[0] for i in Stations_namesLocations])
fig.show()

z= [stn for stn in corr_allStn]
fig = px.imshow(z, template=template, text_auto=True, zmin = 0, zmax = 1.0, title='Correlation',
                labels=dict(y="Station", x='Run', color="Rating"),
                x=[i for i in range(len(Simulation_output_folders))],
                y=[i[0] for i in Stations_namesLocations])
fig.show()